In [1]:
import numpy as np
import pandas as pd
import utils
from nn import NeuralNetwork
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [2]:
def evaluate_classification_metrics(y_true, y_pred, multiclass=False):

    average_type = 'macro' if multiclass else 'binary'

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average=average_type, zero_division=0)
    rec = recall_score(y_true, y_pred, average=average_type, zero_division=0)
    f1 = f1_score(y_true, y_pred, average=average_type, zero_division=0)

    print(f"✅ Accuracy : {acc:.4f}")
    print(f"✅ Precision: {prec:.4f}")
    print(f"✅ Recall   : {rec:.4f}")
    print(f"✅ F1 Score : {f1:.4f}")

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1_score": f1
    }


In [3]:
# Load the dat
train_data = pd.read_csv(
    'data/data.csv'
)

train_data.rename(columns={
    "diagnosis": "class",
    "id": "ID"
}, inplace=True)

train_data = train_data.drop(["Unnamed: 32"],axis=1)

le = LabelEncoder()
train_data['class'] = le.fit_transform(train_data['class'].values.ravel())


In [4]:
train_data.head()

,ID,class,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
# Drop ID column and extract features/labels
X = train_data.drop(columns=["ID", "class"])
y = train_data["class"].astype(int)  # convert True/False to 1/0

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/validation split (since test_data has no labels for now)
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [6]:
configs = [
    # Simple
    {"layers": [30, 1], "act": [utils.sigmoid], "lr": 0.01},
    {"layers": [30, 1], "act": [utils.sigmoid], "lr": 0.1},

    # 1 hidden layer
    {"layers": [30, 16, 1], "act": [utils.relu, utils.sigmoid], "lr": 0.01},
    {"layers": [30, 16, 1], "act": [utils.tanh, utils.sigmoid], "lr": 0.01},
    {"layers": [30, 16, 1], "act": [utils.relu, utils.sigmoid], "lr": 0.1},
    {"layers": [30, 16, 1], "act": [utils.tanh, utils.sigmoid], "lr": 0.1},

    # 2 hidden layers
    {"layers": [30, 16, 8, 1], "act": [utils.relu, utils.relu, utils.sigmoid], "lr": 0.01},
    {"layers": [30, 16, 8, 1], "act": [utils.tanh, utils.relu, utils.sigmoid], "lr": 0.01},
    {"layers": [30, 16, 8, 1], "act": [utils.tanh, utils.tanh, utils.sigmoid], "lr": 0.01},
    {"layers": [30, 16, 8, 1], "act": [utils.relu, utils.relu, utils.sigmoid], "lr": 0.1},
    {"layers": [30, 16, 8, 1], "act": [utils.tanh, utils.relu, utils.sigmoid], "lr": 0.1},
    {"layers": [30, 16, 8, 1], "act": [utils.tanh, utils.tanh, utils.sigmoid], "lr": 0.1}
]


In [ ]:
results = []

for i, cfg in enumerate(configs):
    print(f"\n🔁 Running config {i+1}/{len(configs)}")
    print(f"Layers: {cfg['layers']}, Activations: {[fn.__name__ for fn in cfg['act']]}, LR: {cfg['lr']}")

    nn = NeuralNetwork(cfg["layers"], cfg["act"], learning_rate=cfg["lr"])
    
    # Train including validation set
    nn.train(X_train, y_train.values, epochs=100, X_val=X_val, y_val=y_val.values)
    
    # Predict final output
    preds = nn.predict(X_val)
    metrics = evaluate_classification_metrics(y_val.values, preds)
    
    # Get metrics stored during training from the model
    train_loss = nn.history["train_loss"][-1]
    val_loss = nn.history["val_loss"][-1] if nn.history["val_loss"] else None
    train_acc = nn.history["train_acc"][-1]
    val_acc = nn.history["val_acc"][-1] if nn.history["val_acc"] else None


    results.append({
        "config_id": i + 1,
        "layers": cfg["layers"],
        "activations": [fn.__name__ for fn in cfg["act"]],
        "learning_rate": cfg["lr"],

        # Add train history
        "train_loss": float(train_loss),
        "val_loss": float(val_loss) if val_loss is not None else None,
        "train_acc": float(train_acc),
        "val_acc": float(val_acc) if val_acc is not None else None,

        # Add evaluation metrics from evaluate_classification_metrics()
        **metrics})



In [9]:
df_results = pd.DataFrame(results)
df_results

,config_id,layers,activations,learning_rate,train_loss,val_loss,train_acc,val_acc,accuracy,precision,recall,f1_score
0,1,"[30, 1]",[sigmoid],0.01,0.047236,0.052147,0.991209,0.991228,0.991228,1.000000,0.976744,0.988235
1,2,"[30, 1]",[sigmoid],0.10,0.041354,0.099206,0.986813,0.964912,0.964912,0.933333,0.976744,0.954545
2,3,"[30, 16, 1]","[relu, sigmoid]",0.01,0.002585,0.183646,1.000000,0.938596,0.938596,0.928571,0.906977,0.917647
3,4,"[30, 16, 1]","[tanh, sigmoid]",0.01,0.006536,0.151044,1.000000,0.956140,0.956140,0.931818,0.953488,0.942529
4,5,"[30, 16, 1]","[relu, sigmoid]",0.10,0.000094,0.137942,1.000000,0.973684,0.973684,0.976190,0.953488,0.964706
5,6,"[30, 16, 1]","[tanh, sigmoid]",0.10,0.000486,0.092355,1.000000,0.964912,0.964912,0.933333,0.976744,0.954545
6,7,"[30, 16, 8, 1]","[relu, relu, sigmoid]",0.01,0.001008,0.260621,1.000000,0.956140,0.956140,0.952381,0.930233,0.941176
7,8,"[30, 16, 8, 1]","[tanh, relu, sigmoid]",0.01,0.000915,0.220766,1.000000,0.956140,0.956140,0.952381,0.930233,0.941176
8,9,"[30, 16, 8, 1]","[tanh, tanh, sigmoid]",0.01,0.005248,0.142221,0.997802,0.956140,0.956140,0.913043,0.976744,0.943820
9,10,"[30, 16, 8, 1]","[relu, relu, sigmoid]",0.10,0.665770,0.662950,0.626374,0.622807,0.622807,0.000000,0.000000,0.000000
